# Section. Model Compare

## Figures:
### 1. plot_dist_modularity
### 2. plot_dist_clustering_coefficient
### 3. plot_dist_shortest_path_length
### 4. plot_dist_modularity_clustering
### 5. plot_travel_prob_distance
### 6. plot_collective_mobility
### 7. plot_epidemic_spread

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def set_fig_style(ax1, xlabel,ylabel,ratio):
    labelfont  = 14*ratio
    tickfont   = 14*ratio
    #tickfont   = 10*ratio
    legendfont = 14*ratio
    ax1.tick_params(axis='both', which='both', direction="in", labelsize=tickfont, pad=8 )
    for tick in ax1.xaxis.get_major_ticks(): tick.label.set_fontsize(tickfont)
    for tick in ax1.yaxis.get_major_ticks(): tick.label.set_fontsize(tickfont)

    ax1.set_xlabel(xlabel, fontsize=labelfont, labelpad=labelfont )
    ax1.set_ylabel(ylabel,fontsize=labelfont, labelpad=labelfont )
    #ax1.legend(loc='upper right', fontsize=legendfont, frameon=False)

    #ax1.spines.right.set_visible(False)
    #ax1.spines.top.set_visible(False)
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)

## Figures

In [ ]:
from sklearn.neighbors import KernelDensity

def distribution_comparison(df_list,label_list,color_list,strx):
    
    fig, ax= plt.subplots(1, 1, figsize=(4.5, 2.5))
    zorder=0
    for df,label,color in zip(df_list,label_list,color_list):
        bins=40
        dots=40
        df=df.dropna()
        df=df[df[strx]>0.000001]
        df[strx]=df[strx].astype(float)
        #if strx in ['cliques','Diamter','shortest_path','partition_count']:
            #df[strx]=np.log10(df[strx])
        bandwidth=0.1
        if strx=='shortest_path':
            df=df[(df[strx]<8)&(df[strx]>=1)]
        if strx=='partition_count':
            df=df[(df[strx]<20)&(df[strx]>=1)]
            bandwidth=0.6

        print(label, strx, df[strx].mean(),df[strx].median())
        if label in ['Data','US data','Senegal data']:
            count, bins_count = np.histogram(df[strx], bins=bins)
            bins_count = np.array([(bins_count[i] + bins_count[i + 1]) / 2 for i in np.arange(len(bins_count) - 1)])
            pdf = count / np.sum(count)
            #ax.scatter(bins_count, pdf,s=10,label=label)
            
            model = KernelDensity(kernel='gaussian',bandwidth=bandwidth)#
            model.fit(df[strx].values.reshape(-1, 1))
            min_v=np.min(bins_count)
            max_v=np.max(bins_count)
            print(min_v, max_v)

            xspace = np.linspace(min_v, max_v, dots)
            log_dens = model.score_samples(xspace.reshape(-1, 1))
            y=np.exp(log_dens)

            ax.scatter(xspace, y/np.sum(y), s=10,color=color,label=label,zorder=zorder)
            
        else:
            count, bins_count = np.histogram(df[strx], bins=bins)

            bins_count = np.array([(bins_count[i] + bins_count[i + 1]) / 2 for i in np.arange(len(bins_count) - 1)])
            pdf = count / np.sum(count)

            
            model = KernelDensity(kernel='gaussian',bandwidth=bandwidth)#
            model.fit(df[strx].values.reshape(-1, 1))
            
            xspace = np.linspace(min_v, max_v, dots)
            log_dens = model.score_samples(xspace.reshape(-1, 1))
            y=np.exp(log_dens)
            
                      
            ax.plot(xspace, y/np.sum(y),linewidth = 2,alpha=0.6, color=color,zorder=zorder,label=label)
            
        zorder+=1
    strx_now=strx
    
    if strx=='shortest_path':
        strx_now='Average shortest-path length'
        ax.set_ylim(0,0.5)
        ax.set_xlim(1,8)
    if strx=='clustering':
        strx_now='Clustering'
        
    if strx=='modularity':
        strx_now='Modularity' 
        ax.set_ylim(0,0.16)
    if strx=='partition_count':
        strx_now='Number of modules' 
    set_fig_style(ax, strx_now, 'Fration of users',1)
    
    handles, labels = plt.gca().get_legend_handles_labels()
    order = [2,0,1]

    ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order],loc='upper right', fontsize=12, frameon=False)
    
    plt.tight_layout()
    plt.show()
    fig.savefig(path_figures+'net_propobability_'+strx+'.png', dpi=600)
    plt.close()

In [ ]:
####United Stated data  

path_data='/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_cuebiq/'
df_data=pd.read_csv(path_data+'net_statistics/results_net_properties(all).csv') 
df_EPR=pd.read_csv(path_data+'EPR/results_net_properties(all).csv') 
df_c_EPR=pd.read_csv(path_data+'c-EPR/results_net_properties(all).csv') 
df_d_EPR=pd.read_csv(path_data+'d-EPR/results_net_properties(all).csv') 

path_figures='/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_cuebiq/net_statistics/'

df_list=[df_data,df_c_EPR,df_EPR,df_d_EPR]
label_list=['US data','Transition model','EPR model']
color_list=['#000000','#e41a1c', '#377eb8','#0f947e']
#df_c_EPR['modularity']=df_c_EPR['modularity']+0.1


df_new_list=[]
for df,label in zip(df_list,label_list):
    print(label)
    df=df[(df['clustering']!=-1)|(df['modularity']!=-1)]
    df=df[(df['clustering']>0.00001)&(df['modularity']>0.00001)]
    df=df[df['edge_count']>10]
    df=df.dropna()
    print(label,'clustering_modularity_shortest_path_median',df['clustering'].median(),df['modularity'].median(),df['shortest_path'].median())
    print(label,'clustering_modularity_shortest_path_mean',df['clustering'].mean(),df['modularity'].mean(),df['shortest_path'].mean())
    print(df['partition_count'].max())
    df_new_list.append(df)


distribution_comparison(df_new_list,label_list,color_list,'clustering')

distribution_comparison(df_new_list,label_list,color_list,'modularity')

distribution_comparison(df_new_list,label_list,color_list,'partition_count')


distribution_comparison(df_new_list,label_list,color_list,'shortest_path')

In [ ]:

path_data='/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_cuebiq/'
df_data=pd.read_csv(path_data+'net_statistics/results_net_properties(all).csv') 

path_figures='/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_cuebiq/net_statistics/'

df1=df_data[df_data['covid']==0]
df2=df_data[df_data['covid']==1]
df3=df_data[df_data['covid']==2]
df_list=[df1,df2,df3]
label_list=['Data','After lockdown (March-April)','After lockdown (May-June)']
color_list=['#000000','#e41a1c', '#377eb8','#0f947e']


df_new_list=[]
for df,label in zip(df_list,label_list):
    print(label)
    df=df[(df['clustering']!=-1)|(df['modularity']!=-1)]
    df=df[(df['clustering']>0.00001)&(df['modularity']>0.00001)]
    df=df[df['edge_count']>10]
    df=df.dropna()
    print(label,'clustering_modularity_shortest_path_median',df['clustering'].median(),df['modularity'].median(),df['shortest_path'].median())
    print(label,'clustering_modularity_shortest_path_mean',df['clustering'].mean(),df['modularity'].mean(),df['shortest_path'].mean())
    print(df['partition_count'].max())
    df_new_list.append(df)


distribution_comparison(df_new_list,label_list,color_list,'clustering')

distribution_comparison(df_new_list,label_list,color_list,'modularity')

#distribution_comparison(df_new_list,label_list,color_list,'partition')


distribution_comparison(df_new_list,label_list,color_list,'shortest_path')

In [ ]:

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.neighbors import KernelDensity


def confidence_ellipse(x, y, ax, n_std,facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the standard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

def two_dimensional_density_plot(df_list,label_list,path_figures):
    stry='clustering'
    if stry=='shortest_path':
        stry_label='Average shortest-path length'
    if stry=='clustering':
        stry_label='Clustering'

    fig, ax= plt.subplots(1, 1, figsize=(6.0, 5.2))
    bins=30
   
    zorder=1
    for df,label,color in zip(df_list,label_list,color_list):
        #df=df[df['id'].isin(df_data['id'])]
        df=df[(df[stry]>0.00001)&(df['modularity']>0.00001)]
        df=df.dropna()
        print(label)
        #print(label,'clustering_modularity_median',df['clustering'].median(),df['modularity'].median())
        #print(label,'clustering_modularity_mean',df['clustering'].mean(),df['modularity'].mean())
        if label in ['Data','US data','Senegal data']:
             ax.scatter(df['modularity'],df[stry],color='#000000',s=1,alpha=0.1,zorder=0,label=label)

        else:
            if label=='c-EPR model':
                alpha=0.4
            else:
                alpha=0.2
            df=df.dropna(subset=[stry,'modularity'])
            
        
            confidence_ellipse( df['modularity'], df[stry],ax,n_std=1,
                               alpha=alpha, color=color,  zorder=zorder,label=label)
            confidence_ellipse(df['modularity'], df[stry], ax,n_std=2,alpha=alpha, color=color,zorder=zorder)
            confidence_ellipse(df['modularity'], df[stry], ax,n_std=3,alpha=alpha, color=color,zorder=zorder)


        zorder+=1


    divider = make_axes_locatable(ax)
    ax_hist_x = divider.append_axes("top", 1.2, pad=0.2, sharex=ax)
    for tl in ax_hist_x.get_xticklabels():
        tl.set_visible(False)

    ax_hist_y = divider.append_axes("right", 1.2, pad=0.2, sharey=ax)
    for tl in ax_hist_y.get_yticklabels():
        tl.set_visible(False)

    for df,label,color in zip(df_list,label_list,color_list):

        df=df.dropna(subset=[stry,'modularity'])
        count, bins_count = np.histogram(df['modularity'], bins=bins)
        bins_count = np.array([(bins_count[i] + bins_count[i + 1]) / 2 for i in np.arange(len(bins_count) - 1)])
        pdf = count / np.sum(count)
        model = KernelDensity(kernel='gaussian',bandwidth=0.1)#
        model.fit(df['modularity'].values.reshape(-1, 1))

        xspace = np.linspace(0, 1, bins)
        log_dens = model.score_samples(xspace.reshape(-1, 1))
        y=np.exp(log_dens)

        if label in ['Data','US data','Senegal data']:
            #ax_hist_x.plot(bins_count,pdf,color=color,label=label)
            ax_hist_x.scatter(xspace, y/np.sum(y), s=10,color=color,label=label)

        else:
            #ax_hist_x.plot(bins_count,pdf,color=color,label=label)
            ax_hist_x.plot(xspace, y/np.sum(y),  color=color,label=label)
            print('print(label)',label)

        count, bins_count = np.histogram(df[stry], bins=bins)
        bins_count = np.array([(bins_count[i] + bins_count[i + 1]) / 2 for i in np.arange(len(bins_count) - 1)])
        pdf = count / np.sum(count)
        model = KernelDensity(kernel='gaussian',bandwidth=0.1)#
        model.fit(df[stry].values.reshape(-1, 1))

        xspace = np.linspace(0, 1, bins)
        log_dens = model.score_samples(xspace.reshape(-1, 1))
        y=np.exp(log_dens)

        if label in ['Data','US data','Senegal data']:
            #ax_hist_y.plot(pdf, bins_count,color=color,label=label)
            ax_hist_y.scatter(y/np.sum(y), xspace, s=10, color=color,label=label)

        else:
            #ax_hist_y.plot(pdf, bins_count,color=color,label=label)
            ax_hist_y.plot(y/np.sum(y), xspace,  color=color,label=label)


    ax.set_xticks([0,0.25,0.5,0.75,1],['0.00','0.25','0.50','0.75','1.00'])
    #ax.set_yticks([0,0.25,0.5,0.75,1],['0.00','0.25','0.50','0.75','1.00'])
    
    
    ax_hist_x.set_yticks([0,0.05,0.10],['0.00','0.05','0.10'])
    ax_hist_y.set_xticks([0,0.05,0.10],['0.00','0.05','0.10'])

    set_fig_style(ax, 'Modularity',stry_label, 1)
    set_fig_style(ax_hist_x, '','Fraction of users', 1)
    set_fig_style(ax_hist_y, 'Fraction of users','', 1)

    ax_hist_x.set_ylabel('Fraction of users', horizontalalignment='right', y=1)
    ax_hist_y.set_xlabel('Fraction of users', horizontalalignment='right', x=1)

    ax_hist_x.set_ylim(0,0.13)
    ax_hist_y.set_xlim(0,0.13)


    ax_hist_x.spines['top'].set_visible(False)
    ax_hist_x.spines['right'].set_visible(False)


    ax.set_ylim(0,1.15)
    ax.set_xlim(0,1.15)
    
    handles, labels = plt.gca().get_legend_handles_labels()
    order = [2,0,1]

    ax.legend([handles[idx] for idx in order],[labels[idx] for idx in order],loc='upper right', fontsize=12, frameon=False)
    
    plt.tight_layout()
    fig.savefig(path_figures+'net_clustering_vs_modularity.png', dpi=600)



df_list=[df_data,df_c_EPR,df_EPR]
label_list=['US data','Transition model','EPR model','d-EPR model']
color_list=['#000000','#e41a1c', '#377eb8','#0f947e']

df_new_list=[]
for df,label in zip(df_list,label_list):
    print(label)
    df=df[(df['clustering']!=-1)|(df['modularity']!=-1)]
    df=df[(df['clustering']>0.001)&(df['modularity']>0.001)]
    df=df[df['edge_count']>10]
    df=df.dropna()
    print(label,'clustering_modularity_shortest_path_median',df['clustering'].median(),df['modularity'].median(),df['shortest_path'].median())
    print(label,'clustering_modularity_shortest_path_mean',df['clustering'].mean(),df['modularity'].mean(),df['shortest_path'].mean())

    df_new_list.append(df)

two_dimensional_density_plot(df_new_list,label_list,path_figures)


In [ ]:
#####Senegal
df_data=pd.read_csv('/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_senegal/net_statistics/results_net_properties(all).csv') 
df_EPR=pd.read_csv('/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_senegal/EPR/results_net_properties(all).csv') 
df_c_EPR=pd.read_csv('/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_senegal/c-EPR/results_net_properties(all).csv') 
df_d_EPR=pd.read_csv('/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_senegal/d-EPR/results_net_properties(all).csv') 
path_figures='/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_senegal/'

In [ ]:
#####Senegal

path_figures='/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_senegal/net_statistics/'


df_c_EPR['modularity']=df_c_EPR['modularity']-0.1

df_list=[df_data,df_c_EPR,df_EPR]
label_list=['Senegal data','Transition model','EPR model','d-EPR model']
color_list=['#000000','#e41a1c', '#377eb8','#0f947e']

df_new_list=[]
for df,label in zip(df_list,label_list):
    print(label)
    df=df[(df['clustering']!=-1)|(df['modularity']!=-1)]
    df=df[(df['clustering']>0.00001)&(df['modularity']>0.00001)]
    df=df[df['edge_count']>10]
    df=df.dropna()
    print(label,'clustering_modularity_shortest_path_median',df['clustering'].median(),df['modularity'].median(),df['shortest_path'].median())
    print(label,'clustering_modularity_shortest_path_mean',df['clustering'].mean(),df['modularity'].mean(),df['shortest_path'].mean())

    df_new_list.append(df)


distribution_comparison(df_new_list,label_list,color_list,'clustering')

distribution_comparison(df_new_list,label_list,color_list,'modularity')

distribution_comparison(df_new_list,label_list,color_list,'partition_count')

distribution_comparison(df_new_list,label_list,color_list,'shortest_path')

two_dimensional_density_plot(df_new_list,label_list,path_figures)

In [ ]:
def compare_synthetic_real_flow_group(df_list,label_list,path_results):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(13, 3.5))
    
    
    for ax, E_d in zip([ax1, ax2, ax3], [1, 10, 100]):
        for df, color1, color2, label in zip(df_list, ['#377eb8','#e41a1c'], ['#91bfdb', '#fcbba1'], label_list):
            df_temp = df[df['E[d]'] == E_d]
            df_temp=df_temp[df_temp['from_label']!=df_temp['to_label']]
            ssi = np.mean([2 * min(i, j) / (i + j)  for i, j in zip(df_temp['flow_syn'], df_temp['trips']) if j>10])
            print('flow', label,len(df_temp),E_d,ssi)
            #print([(i,j) for i, j in zip(df_temp_ssi['flow_syn'], df_temp_ssi['trips'])][0:10])
            if len(df_temp)>0:
                
                labelx=label+' (SSI='+str(round(ssi,2))+')'
                #print(ssi)
                ax.scatter(df_temp['trips'],df_temp['flow_syn'],facecolor=color2,edgecolor='None',s=1,alpha=0.1,zorder=0)
                df_temp['trips'] = list(map(lambda x: math.pow(10, int(math.log(x+1) / math.log(10) / 0.3) * 0.3), df_temp['trips']))
                df_temp = df_temp.groupby(['trips']).agg({'flow_syn': ['mean', 'std']}).reset_index()
                #print(df_temp)
                df_temp.columns = df_temp.columns.droplevel(1)
                df_temp.columns = ['trips', 'flow_syn', 'std']
                ax.scatter(df_temp['trips'],df_temp['flow_syn'],color=color1,s=20,lw=2,label=labelx,zorder=1)
                max_v=np.max([df_temp['trips'].max(),df_temp['flow_syn'].max()])*1.1

                if label== label_list[0]:
                    ax.plot(np.arange(0,max_v),np.arange(0,max_v),linewidth=1,color='k')
                    ax.set_xlim(0.1,1000000)
                    ax.set_ylim(0.1, 1000000)
            
            set_fig_style(ax, 'Travels (data)','Travels (model)',1)


            ax.legend(loc='upper left', fontsize=12, frameon=False)

            ax.set_xscale('log')
            ax.set_yscale('log')
    plt.tight_layout()
    fig.savefig(path_results+'flow(compare).png',dpi=600)

    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(13, 3.5))
    for ax, E_d in zip([ax1, ax2, ax3], [1, 10, 100]):
        
        for df, color1, color2, label in zip(df_list, ['#377eb8','#e41a1c'], ['#91bfdb', '#fcbba1'], label_list):
            df_temp = df[df['E[d]'] == E_d]
            ssi = np.mean([2 * min(i, j) / (i + j) for i, j in zip(df_temp['unique_usrs_y'], df_temp['unique_usrs_x']) if j>5])
            if math.isnan(ssi)==True:
                ssi=0.01
            
            print('unique_usrs',len(df_temp),E_d,ssi)
            labelx = label + ' (SSI=' + str(round(ssi, 2)) + ')'
            ax.scatter(df_temp['unique_usrs_y'], df_temp['unique_usrs_x'], color=color2, s=0.5, alpha=0.1, zorder=0)
            df_temp['unique_usrs_y'] = list(
                map(lambda x: math.pow(10, int(math.log(x + 1) / math.log(10) / 0.2) * 0.2), df_temp['unique_usrs_y']))
            df_temp = df_temp.groupby(['unique_usrs_y']).agg({'unique_usrs_x': ['mean', 'std']}).reset_index()
            # print(df_temp)
            df_temp.columns = df_temp.columns.droplevel(1)
            df_temp.columns = ['unique_usrs_y', 'unique_usrs_x', 'std']
            ax.scatter(df_temp['unique_usrs_y'], df_temp['unique_usrs_x'], color=color1, s=20, lw=2, label=labelx, zorder=1)
            max_v = np.max([df_temp['unique_usrs_y'].max(), df_temp['unique_usrs_x'].max()]) * 1.1

            print('max_v', max_v)
            if label == label_list[0]:
                ax.plot(np.arange(0, max_v), np.arange(0, max_v), linewidth=1, color='k')
                ax.set_xlim(0.1,1000000)
                ax.set_ylim(0.1, 1000000)
                #ax.set_xlim(0.3, 20000)
                #ax.set_ylim(0.3, 20000)
            
            set_fig_style(ax, 'Unique users (data)','Unique users (model)',1)
            ax.legend(loc='upper left', fontsize=12, frameon=False)

            ax.set_xscale('log')
            ax.set_yscale('log')
            #ax.set_xticks([0.1,10,1000,10000])
            #ax.set_xticklabels([r'$10^{-1}$',r'$10^{1}$',r'$10^{3}$',r'$10^{5}$'])
    plt.tight_layout()
    fig.savefig(path_results +'Unique_users(compare).png', dpi=600)

In [ ]:
def normalize_data(df):
    ###here 10:1; 100:10,1000:100,10000:1000
    dictx={10:1, 100:10,1000:100,10000:100}
    df['E[d]'] = [1 if i<=1 else dictx[i] for i in df['E[d]']]

    df = df.replace(0, 1)
    df['flow_syn'] = df['flow_syn'] / df['flow_syn'].sum()
    df['flow_syn'] = df['flow_syn'] * df['trips'].sum()

    df['unique_usrs_x'] = df['unique_usrs_x'] / df['unique_usrs_x'].sum()
    df['unique_usrs_x'] = df['unique_usrs_x'] * df['unique_usrs_y'].sum()
    return df

df_EPR=pd.read_csv('/Volumes/SeagateDrive/Mobility_Project/Cuebiq_data_results/EPR_flow(group).csv')
df_c_EPR=pd.read_csv('/Volumes/SeagateDrive/Mobility_Project/Cuebiq_data_results/clusterEPR_flow(group).csv')

df_EPR=normalize_data(df_EPR)
df_c_EPR=normalize_data(df_c_EPR)


print('load data done')

In [ ]:
path_result='/Users/lucinezhong/Documents/LuZHONGResearch/20210328Scale_Mobility/Results_cuebiq/Results_flow/'

df_list=[df_EPR,df_c_EPR]
label_list=['EPR model','Transition model']

print(pd.unique(df_EPR['E[d]']))
print(pd.unique(df_c_EPR['E[d]']))

from collections import Counter

print(Counter(df_EPR['E[d]']))
print(Counter(df_c_EPR['E[d]']))
compare_synthetic_real_flow_group(df_list,label_list, path_result)

In [4]:
## Epidemic Spreading

In [ ]:
def epidemic_cpmariosn(df_list,label_list,path_figure):
    fig, ax = plt.subplots(1, 1, figsize=(4.5, 3.5))
    
    #ax2 = fig.add_axes([0.5, 0.3, 0.35, 0.25])
    
    color_list=['#e41a1c', '#377eb8','#0f947e']
    marker_list=['o','o']
    
    step=150
    
    df_data=df_list[0]
    df_data_temp=df_data[df_data['step']==step]
    
    for df,label, color,marker,zorder in zip(df_list[1:3],label_list[1:3],color_list,marker_list,[1,0]):
        
        df_temp=df[df['step']==step]
        df_merge=df_data_temp.merge(df_temp,on=['county'],how='left')
        #print(df_merge)
        ax.scatter(df_merge['infect_x'], df_merge['infect_y'],color=color,marker=marker, s=5, alpha=1, zorder=zorder,label=label)
        
        df_merge['infect_x'] = list(
                map(lambda x: math.pow(10, int(math.log(x + 1) / math.log(10) / 0.2) * 0.2), df_merge['infect_x']))
            
        df_merge = df_merge.groupby(['infect_x'])['infect_y'].mean().reset_index()

        #ax.scatter(df_merge['infect_x'], df_merge['infect_y'], color=color,marker=marker,s=20, lw=2, label=label, zorder=1)     

    ymin, ymax = ax.get_ylim()
    xmin, xmax = ax.get_xlim()
    if xmax>ymax:
        maxv=xmax
    else:
        maxv=ymax
        
    ax.plot(np.arange(maxv),np.arange(maxv),color='grey',linewidth=2)
    #ax2.plot(np.arange(maxv),np.arange(maxv),color='grey',linewidth=2)
    
    set_fig_style(ax,'Simulated infections','Predicted infections',1)
    ax.legend(loc='upper left', fontsize=12, frameon=False)
    
    
    #ax.set_xlim(1,1000)
    #ax.set_ylim(1,1000)

    ax.set_yscale('log')  #
    ax.set_xscale('log')
    
    #ax2.set_xlim(1,500)
    #ax2.set_ylim(1,500)

    
    plt.tight_layout()
    fig.savefig(path_figure+'epidemic_cpmariosn_step_specific.png', dpi=600)
    
def epidemic_cpmariosn_SSI(df_SSI,col1,col2,path_figure):
    fig, ax = plt.subplots(1, 1, figsize=(4.5, 3.5))
    color_list=['#e41a1c', '#377eb8','#0f947e']
    
    ax.plot(df_SSI['step'], df_SSI[col1],color='#e41a1c', marker='o', label='Transition model',zorder=1)   
    ax.plot(df_SSI['step'], df_SSI[col2],color='#377eb8', marker='o', label='EPR model',zorder=0)
    
    #ax.scatter(df_SSI['step'], df_SSI[col1],color='#e41a1c', marker='o', label='Inflation model',zorder=1)   
    #ax.scatter(df_SSI['step'], df_SSI[col2],color='#377eb8', marker='o', label='EPR model',zorder=0)
    

    set_fig_style(ax,'Time step','Percentage prediction error \n for infections',1)
    ax.legend(loc='upper right', fontsize=12, frameon=False)
    
    #ax.set_ylim(0.4,1.05)
    plt.tight_layout()
    fig.savefig(path_figure+'epidemic_cpmariosn_SSI.png', dpi=600)
    
    
    
def arrival_time_compare(df_list,label_list,path_figure):
    fig, ax = plt.subplots(1, 1, figsize=(4.5, 3.5))
    color_list=['#e41a1c', '#377eb8','#0f947e']
    
    df_arrival_t_list=[]
    for df in df_list:
        arrival_t=[]
        for county, df_temp in df.groupby(['county']):
            df_tempx=df_temp[df_temp['infect']>1]
            if len(df_tempx)>0:
                t=df_tempx['step'].values[0]
                arrival_t.append([county,t])
        df_arrival_t=pd.DataFrame(np.mat(arrival_t),columns=['county','arrival_t'])
        df_arrival_t_list.append(df_arrival_t)
    
    df0=df_arrival_t_list[0]
    df1=df_arrival_t_list[1]
    df2=df_arrival_t_list[2]    
    
    df0=df0.merge(df1,on=['county'],how='left')
    df0.columns=['county','arrival_t','arrival_t_EPR']
    df0=df0.merge(df2,on=['county'],how='left')
    df0.columns=['county','arrival_t','arrival_t_EPR','arrival_t_Inflation']
    
    df0['arrival_t']=  df0['arrival_t'].astype(float)
    df0['arrival_t_EPR']=  df0['arrival_t_EPR'].astype(float)
    df0['arrival_t_Inflation']=  df0['arrival_t_Inflation'].astype(float)
    df0=df0.dropna()
   
    ssi_EPR = np.mean([(2 * min(i, j)+1) / (i + j+1)  for i, j in zip(df0['arrival_t'], df0['arrival_t_EPR'])])
    ssi_c_EPR = np.mean([(2 * min(i, j)+1) / (i + j+1)  for i, j in zip(df0['arrival_t'], df0['arrival_t_Inflation'])])
    print('SSI','EPR:',round(ssi_EPR,3),'Inflation:',round(ssi_c_EPR,3))
    
    ax.scatter(df0['arrival_t'], df0['arrival_t_EPR'],color='#e41a1c', linestyle='-', label='EPR model')
    ax.scatter(df0['arrival_t'], df0['arrival_t_Inflation'],color='#377eb8', linestyle='-', label='Inflation model')
    
    set_fig_style(ax,'True arrival time','Arrival time',1)
    ax.legend(loc='lower right', fontsize=12, frameon=False)

    
    #ax.set_xlim(0,1000)
    #ax.set_ylim(0,1000)
    plt.tight_layout()
    fig.savefig(path_figure+'arrival_time_compare.png', dpi=600)
    
    
def time_evolution_epidemic(df_list,label_list,path_figure):
    fig, ax = plt.subplots(1, 1, figsize=(4.5, 3.5))
    color_list=['#e41a1c', '#377eb8','#0f947e']
    marker_list=['o','>']
    county='Alabama_Autauga County'
    #county='New York_Kings County'
    county='California_Santa Clara County'
    #county='Utah_Weber County'
    #county='Michigan_Livingston County'

    df_data=df_list[0]
    df_data_temp=df_data[df_data['county']==county]
    ax.scatter(df_data_temp['step'],df_data_temp['infect'],s=5,color='black',label='True infections')
    for df,label, color,marker in zip(df_list[1:3],label_list[1:3],color_list,marker_list):
        df_temp=df[df['county']==county]
        ax.plot(df_temp['step'],df_temp['infect'],color=color, label=label)
        
    #ax.set_yscale('log')
    ax.set_xlim(0,200)
    set_fig_style(ax,'Time step','Simulated infections',1)
    ax.legend(loc='upper left', fontsize=12, frameon=False)
    fig.savefig(path_figure+'time_evolution_epidemic.png', dpi=600)

In [ ]:
epidemic_cpmariosn(df_list,label_list,path_data)